In [1]:
years = list(range(1991, 2024))

In [2]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [4]:
import requests
import time
from time import sleep

for year in years:
    url = url_start.format(year)
    sleep(10)
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [3]:
from bs4 import BeautifulSoup

In [8]:
import pandas as pd

In [23]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_="over_header").decompose()
    mvp_table = soup.find(id="mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["Year"] = year
    
    dfs.append(mvp)
    
    

In [24]:
mvps = pd.concat(dfs)

In [61]:
mvps

Rank           Player  Age   Tm  First  Pts Won  Pts Max  Share   G    MP  \
0     1   Michael Jordan   27  CHI   77.0    891.0      960  0.928  82  37.0   
1     2    Magic Johnson   31  LAL   10.0    497.0      960  0.518  79  37.1   
2     3   David Robinson   25  SAS    6.0    476.0      960  0.496  82  37.7   
3     4  Charles Barkley   27  PHI    2.0    222.0      960  0.231  67  37.3   
4     5      Karl Malone   27  UTA    0.0    142.0      960  0.148  82  40.3   
..  ...              ...  ...  ...    ...      ...      ...    ...  ..   ...   
8     9    Stephen Curry   34  GSW    0.0      5.0     1000  0.005  56  34.7   
9    10     Jimmy Butler   33  MIA    0.0      3.0     1000  0.003  64  33.4   
10   11     De'Aaron Fox   25  SAC    0.0      2.0     1000  0.002  73  33.4   
11  12T    Jalen Brunson   26  NYK    0.0      1.0     1000  0.001  68  35.0   
12  12T        Ja Morant   23  MEM    0.0      1.0     1000  0.001  61  31.9   

    ...   TRB   AST  STL  BLK    FG%    3P%    FT%    WS  WS/48  Year  
0   ...   6.0   5.5  2.7  1.0  0.539  0.312  0.851  20.3  0.321  1991  
1   ...   7.0  12.5  1.3  0.2  0.477  0.320  0.906  15.4  0.251  1991  
2   ...  13.0   2.5  1.5  3.9  0.552  0.143  0.762  17.0  0.264  1991  
3   ...  10.1   4.2  1.6  0.5  0.570  0.284  0.722  13.4  0.258  1991  
4   ...  11.8   3.3  1.1  1.0  0.527  0.286  0.770  15.5  0.225  1991  
..  ...   ...   ...  ...  ...    ...    ...    ...   ...    ...   ...  
8   ...   6.1   6.3  0.9  0.4  0.493  0.427  0.915   7.8  0.192  2023  
9   ...   5.9   5.3  1.8  0.3  0.539  0.350  0.850  12.3  0.277  2023  
10  ...   4.2   6.1  1.1  0.3  0.512  0.324  0.780   7.4  0.146  2023  
11  ...   3.5   6.2  0.9  0.2  0.491  0.416  0.829   8.7  0.175  2023  
12  ...   5.9   8.1  1.1  0.3  0.466  0.307  0.748   6.0  0.148  2023  

[499 rows x 21 columns]

In [28]:
mvps.to_csv("mvps.csv")

In [29]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"


In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [41]:
browser = webdriver.Safari()

In [48]:
for year in years:
    url = player_stats_url.format(year)
    time.sleep(10)
    
    browser.get(url)
    browser.execute_script("window.scrollTo(1, 10000)")
    time.sleep(5)

    html = browser.page_source
    
    with open("player/{}.html".format(year), "w+") as f:
        f.write(html)

In [56]:
dfs = []
for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()

    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_="thead").decompose()
    player_table = soup.find(id="per_game_stats")
    player = pd.read_html(str(player_table))[0]
    player["Year"] = year
    dfs.append(player)

In [57]:
player.head()

Rk            Player Pos Age   Tm   G  GS    MP   FG   FGA  ...  ORB  DRB  \
0  1  Precious Achiuwa   C  23  TOR  55  12  20.7  3.6   7.3  ...  1.8  4.1   
1  2      Steven Adams   C  29  MEM  42  42  27.0  3.7   6.3  ...  5.1  6.5   
2  3       Bam Adebayo   C  25  MIA  75  75  34.6  8.0  14.9  ...  2.5  6.7   
3  4      Ochai Agbaji  SG  22  UTA  59  22  20.5  2.8   6.5  ...  0.7  1.3   
4  5      Santi Aldama  PF  22  MEM  77  20  21.8  3.2   6.8  ...  1.1  3.7   

    TRB  AST  STL  BLK  TOV   PF   PTS  Year  
0   6.0  0.9  0.6  0.5  1.1  1.9   9.2  2023  
1  11.5  2.3  0.9  1.1  1.9  2.3   8.6  2023  
2   9.2  3.2  1.2  0.8  2.5  2.8  20.4  2023  
3   2.1  1.1  0.3  0.3  0.7  1.7   7.9  2023  
4   4.8  1.3  0.6  0.6  0.8  1.9   9.0  2023  

[5 rows x 31 columns]

In [58]:
players = pd.concat(dfs)

In [60]:
players

Rk              Player Pos Age   Tm   G  GS    MP   FG   FGA  ...  ORB  \
0      1      Alaa Abdelnaby  PF  22  POR  43   0   6.7  1.3   2.7  ...  0.6   
1      2  Mahmoud Abdul-Rauf  PG  21  DEN  67  19  22.5  6.2  15.1  ...  0.5   
2      3          Mark Acres   C  28  ORL  68   0  19.3  1.6   3.1  ...  2.1   
3      4       Michael Adams  PG  28  DEN  66  66  35.5  8.5  21.5  ...  0.9   
4      5        Mark Aguirre  SF  31  DET  78  13  25.7  5.4  11.7  ...  1.7   
..   ...                 ...  ..  ..  ...  ..  ..   ...  ...   ...  ...  ...   
699  535      Thaddeus Young  PF  34  TOR  54   9  14.7  2.0   3.7  ...  1.3   
700  536          Trae Young  PG  24  ATL  73  73  34.8  8.2  19.0  ...  0.8   
701  537      Omer Yurtseven   C  24  MIA   9   0   9.2  1.8   3.0  ...  0.9   
702  538         Cody Zeller   C  30  MIA  15   2  14.5  2.5   3.9  ...  1.7   
703  539         Ivica Zubac   C  25  LAC  76  76  28.6  4.3   6.8  ...  3.1   

     DRB  TRB   AST  STL  BLK  TOV   PF   PTS  Year  
0    1.4  2.1   0.3  0.1  0.3  0.5  0.9   3.1  1991  
1    1.3  1.8   3.1  0.8  0.1  1.6  2.2  14.1  1991  
2    3.2  5.3   0.4  0.4  0.4  0.6  3.2   4.2  1991  
3    3.0  3.9  10.5  2.2  0.1  3.6  2.5  26.5  1991  
4    3.1  4.8   1.8  0.6  0.3  1.6  2.7  14.2  1991  
..   ...  ...   ...  ...  ...  ...  ...   ...   ...  
699  1.8  3.1   1.4  1.0  0.1  0.8  1.6   4.4  2023  
700  2.2  3.0  10.2  1.1  0.1  4.1  1.4  26.2  2023  
701  1.7  2.6   0.2  0.2  0.2  0.4  1.8   4.4  2023  
702  2.6  4.3   0.7  0.2  0.3  0.9  2.2   6.5  2023  
703  6.8  9.9   1.0  0.4  1.3  1.5  2.9  10.8  2023  

[19589 rows x 31 columns]

In [ ]:
players.to_csv("players.csv")